In [5]:
import pandas as pd  #import dependencies
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

#set visualization styles of this
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12,8)

In [14]:
#Load dataset form Hugging Face
print("Loading medical tanscriptions - dataset !")
dataset = load_dataset("hpe-ai/medical-cases-classification-tutorial")
print(dataset)

#convert to the pandas dataframes - (training ,validations and testing )
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

print("Dataset Loaded sucessfuly ✅!")

print(f"Training set :{len(train_df)}")
print(f"Validation set:{len(val_df)}")
print(f"test set:{len(val_df)}")

print(f"Total sets:{len(train_df)+len(val_df)+len(test_df)}")



Loading medical tanscriptions - dataset !


Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['description', 'transcription', 'sample_name', 'medical_specialty', 'keywords'],
        num_rows: 1724
    })
    validation: Dataset({
        features: ['description', 'transcription', 'sample_name', 'medical_specialty', 'keywords'],
        num_rows: 370
    })
    test: Dataset({
        features: ['description', 'transcription', 'sample_name', 'medical_specialty', 'keywords'],
        num_rows: 370
    })
})
Dataset Loaded sucessfuly ✅!
Training set :1724
Validation set:370
test set:370
Total sets:2464


In [18]:
#Display basic information about this dataset
print("Dataset stucture :")
print(train_df.info())
print("\n display first few rows :")
print(train_df.head())

Dataset stucture :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1724 entries, 0 to 1723
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        1724 non-null   object
 1   transcription      1724 non-null   object
 2   sample_name        1724 non-null   object
 3   medical_specialty  1724 non-null   object
 4   keywords           1109 non-null   object
dtypes: object(5)
memory usage: 67.5+ KB
None

 display first few rows :
                                         description  \
0  Pacemaker ICD interrogation.  Severe nonischem...   
1  Erythema of the right knee and leg, possible s...   
2  Left cardiac catheterization with selective ri...   
3  Patient with a history of coronary artery dise...   
4  Cardiac evaluation and treatment in a patient ...   

                                       transcription  \
0  PROCEDURE NOTE: , Pacemaker ICD interrogation....   
1  PREOPERATIVE DIAGNOSE

In [19]:
#Column anlysis and their data types 
print("column Anlysis .!")
for col in train_df.columns:
    print(f"\n{col}:")
    print(f"  Data type: {train_df[col].dtype}")
    print(f"  Non-null values: {train_df[col].count()}/{len(train_df)}")
    if train_df[col].dtype == 'object':
        print(f"  Unique values: {train_df[col].nunique()}")
        if col == 'medical_specialty':
            print(f"  Categories: {sorted(train_df[col].unique())}")

column Anlysis .!

description:
  Data type: object
  Non-null values: 1724/1724
  Unique values: 1259

transcription:
  Data type: object
  Non-null values: 1724/1724
  Unique values: 1263

sample_name:
  Data type: object
  Non-null values: 1724/1724
  Unique values: 1294

medical_specialty:
  Data type: object
  Non-null values: 1724/1724
  Unique values: 13
  Categories: ['Cardiovascular / Pulmonary', 'ENT - Otolaryngology', 'Gastroenterology', 'Hematology - Oncology', 'Nephrology', 'Neurology', 'Neurosurgery', 'Obstetrics / Gynecology', 'Ophthalmology', 'Orthopedic', 'Pediatrics - Neonatal', 'Psychiatry / Psychology', 'Radiology']

keywords:
  Data type: object
  Non-null values: 1109/1724
  Unique values: 971
